In [1]:
#coding=utf-8
import os
import torch
from torch import nn,optim
import torch.nn.functional as F
#from torch.autograd import Variable
from torchvision import datasets, transforms

In [2]:
#transforms.Compose设置数据预处理方式，
transform = transforms.Compose([
                       transforms.ToTensor(),    #数据转化为tensor对象
                       transforms.Normalize((0.1307,), (0.3081,))   #数据归一化处理
                   ])
#下载mnist训练和测试数据集
trainset = datasets.MNIST('data', train=True, download=True, transform=transform)
testset = datasets.MNIST('data', train=False, download=True, transform=transform)

In [3]:
class LeNet(nn.Module):
    # 定义Net的初始化函数，本函数定义了神经网络的基本结构
    def __init__(self):
        # 继承父类的初始化方法，即先运行nn.Module的初始化函数
        super(LeNet,self).__init__()
        # C1卷积层：输入1张灰度图片，输出6张特征图（即有6种卷积核），卷积核5x5
        self.c1 = nn.Conv2d(1,6,(5,5))
        # C3卷积层：输入6张特征图，输出16张特征图，卷积核5x5（可简化成5）
        self.c3 = nn.Conv2d(6,16,5)
        # 全连接层S4->C5：从S4到C5是全连接，S4层中16*4*4个节点全连接到C5层的120个节点上
        self.fc1 = nn.Linear(16*4*4,120)
        # 全连接层C5->F6：C5层的120个节点全连接到F6的84个节点上
        self.fc2 = nn.Linear(120,84)
        # 全连接层F6->OUTPUT：F6层的84个节点全连接到OUTPUT层的10个节点上，10个节点的输出代表着0到9的不同分值。
        self.fc3 = nn.Linear(84,10)

    # 定义向前传播函数
    def forward(self,x):
        # 输入的灰度图片x经过c1的卷积之后得到6张特征图，然后使用relu函数，增强网络的非线性拟合能力，接着使用2x2窗口的最大池化，然后更新到x
        x = F.max_pool2d(F.relu(self.c1(x)),2)
        # 输入x经过c3的卷积之后由原来的6张特征图变成16张特征图，经过relu函数，并使用最大池化后将结果更新到x
        x = F.max_pool2d(F.relu(self.c3(x)),2)
        # 使用view函数将张量x（S4）变形成一维向量形式，总特征数不变，为全连接层做准备
        x = x.view(-1,self.num_flat_features(x))   #自定义的num_flat_features（）函数，可计算张量x的总特征量
        # 输入S4经过全连接层fc1，再经过relu，更新到x
        x = F.relu(self.fc1(x))
        # 输入C5经过全连接层fc2，再经过relu，更新到x
        x = F.relu(self.fc2(x))
        # 输入F6经过全连接层fc3，更新到x
        x = self.fc3(x)
        return x
    # 计算张量x的总特征量
    def num_flat_features(self,x):
        # 由于默认批量输入，第零维度的batch剔除
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [4]:
CUDA = torch.cuda.is_available()
if CUDA:
    lenet = LeNet().cuda()
else:
    lenet = LeNet()
params = list(lenet.parameters())
print(len(params))
for param in params:
    print(param.size())


10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 256])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [5]:
#损失函数为交叉熵函数
criterion=nn.CrossEntropyLoss()
#优化器为随机梯度下降
optimizer = optim.SGD(lenet.parameters(),lr=0.001,momentum=0.9)


In [6]:
#加载训练和测试数据。batch_size是一次加载的数据量，shuffle遍历不同批次的数据时是否打乱顺序，num_workers使用的子进程数
trainloader = torch.utils.data.DataLoader(trainset,batch_size=4, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset,batch_size=4, shuffle=False, num_workers=2)

In [7]:
def train(model,criterion,optimizer,epochs=1):
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader,0):
            inputs,labels = data
            if CUDA:
                inputs,labels = inputs.cuda(),labels.cuda()
            optimizer.zero_grad()
            #使用网络模型计算输出
            outputs = model(inputs)
            #对比输出结果和标签，得出损失值
            loss = criterion(outputs,labels)
            #反向传播，得到每个叶子节点的梯度
            loss.backward()
            #使用随机梯度下降的方法，逐渐更新参数，以减小梯度。
            optimizer.step()
            #当前损失值
            running_loss += loss.item()  
            
            if i%1000==999:
                print('[Epoch:%d, Batch:%5d] Loss: %.3f' % (epoch+1, i+1, running_loss / 1000))
                running_loss = 0.0

    print('Finished Training')

In [8]:
#train(lenet,criterion,optimizer,epochs=2)

In [9]:
#测试程序，使用训练好的模型对测试数据进行计算，对比输出结果和标签，统计正确率
def test(testloader,model):
    correct = 0   #正确总数
    total = 0     #labels总数
    for data in testloader:
        images, labels = data
        if CUDA:
            images = images.cuda()
            labels = labels.cuda()
        outputs = model(images)   #计算结果
        _, predicted = torch.max(outputs.data, 1)    #找出每组10个输出值总最大的那个值，即预测的结果（数字几）
        total += labels.size(0)
        correct += (predicted == labels).sum()
    print('Accuracy on the test set: %d %%' % (100 * correct / total))

In [10]:
#构造加载和保存模型参数的函数
def load_param(model,path):
    if os.path.exists(path):
        model.load_state_dict(torch.load(path))

def save_param(model,path):
    torch.save(model.state_dict(),path)

In [42]:
load_param(lenet,'model.pkl')

train(lenet,criterion,optimizer,epochs=2)

save_param(lenet,'model.pkl')
print(lenet)
test(testloader,lenet)

[Epoch:1, Batch: 1000] Loss: 0.025
[Epoch:1, Batch: 2000] Loss: 0.028
[Epoch:1, Batch: 3000] Loss: 0.025
[Epoch:1, Batch: 4000] Loss: 0.027
[Epoch:1, Batch: 5000] Loss: 0.025
[Epoch:1, Batch: 6000] Loss: 0.023
[Epoch:1, Batch: 7000] Loss: 0.038
[Epoch:1, Batch: 8000] Loss: 0.028
[Epoch:1, Batch: 9000] Loss: 0.025
[Epoch:1, Batch:10000] Loss: 0.025
[Epoch:1, Batch:11000] Loss: 0.022
[Epoch:1, Batch:12000] Loss: 0.033
[Epoch:1, Batch:13000] Loss: 0.026
[Epoch:1, Batch:14000] Loss: 0.030
[Epoch:1, Batch:15000] Loss: 0.036
[Epoch:2, Batch: 1000] Loss: 0.015
[Epoch:2, Batch: 2000] Loss: 0.028
[Epoch:2, Batch: 3000] Loss: 0.016
[Epoch:2, Batch: 4000] Loss: 0.032
[Epoch:2, Batch: 5000] Loss: 0.027
[Epoch:2, Batch: 6000] Loss: 0.028
[Epoch:2, Batch: 7000] Loss: 0.024
[Epoch:2, Batch: 8000] Loss: 0.029
[Epoch:2, Batch: 9000] Loss: 0.022
[Epoch:2, Batch:10000] Loss: 0.018
[Epoch:2, Batch:11000] Loss: 0.025
[Epoch:2, Batch:12000] Loss: 0.022
[Epoch:2, Batch:13000] Loss: 0.020
[Epoch:2, Batch:1400

In [45]:
print(list(lenet.parameters()))

[Parameter containing:
tensor([[[[-5.6857e-03, -3.1990e-02,  2.5046e-01,  4.7482e-01, -2.7691e-01],
          [ 2.7009e-01, -3.7381e-02,  6.0075e-02,  6.5014e-01,  8.1837e-02],
          [-8.6480e-02, -1.4869e-01, -1.0861e-01,  4.9632e-01,  3.2693e-01],
          [-3.6220e-01, -2.2921e-01, -4.3732e-01, -1.2360e-01,  1.3620e-01],
          [-1.9484e-01, -2.9600e-01, -4.5035e-01, -2.8828e-01,  9.2045e-03]]],


        [[[-2.7453e-01, -2.8823e-01,  1.6204e-01,  1.9460e-01, -2.9833e-02],
          [-1.4568e-01, -2.8488e-01,  1.1675e-01,  2.1153e-01, -2.9295e-02],
          [-2.5549e-01, -2.1471e-01, -7.0573e-02,  3.5597e-01, -1.4938e-01],
          [-1.2872e-01, -7.7373e-02, -4.1098e-02,  3.0790e-01,  5.0315e-02],
          [-2.1182e-01,  9.4348e-02,  1.9119e-01,  7.2459e-02,  1.5661e-01]]],


        [[[-6.6125e-02, -1.4872e-02,  1.3371e-01,  1.7488e-01, -3.2229e-01],
          [ 4.4674e-03,  9.7805e-02,  3.2768e-01,  2.2594e-01, -2.6777e-01],
          [-9.8161e-02, -2.2150e-01,  3.3783e